In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
google_api_key = os.getenv("GOOGLE_API_KEY")

In [6]:
if google_api_key=="":
    print("api key not found")
else:
    print("api key found")

api key found


In [26]:
from llama_index.llms.gemini import Gemini
import google.generativeai as genai

from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from IPython.display import Markdown, display

from llama_index.core import Settings

from llama_index.core import StorageContext, load_index_from_storage

from llama_index.embeddings.gemini import GeminiEmbedding

#### configuring model

In [9]:
genai.configure(api_key=google_api_key)

In [10]:
for models in genai.list_models():
    print(models)

Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representatio

In [12]:
for models in genai.list_models():
    if 'generateContent' in models.supported_generation_methods:
        print(models.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827


In [16]:
documents=SimpleDirectoryReader("../Data")

### loading data

In [20]:
doc = documents.load_data()

In [21]:
doc[0].text

'The Indian Premier League (IPL), also known as the TATA IPL for sponsorship reasons, is a men\'s Twenty20 (T20) cricket league held annually in India. Founded by the BCCI (the Board of Control for Cricket in India) in 2007, the league features ten state or city-based franchise teams.[3][4] Most popular and biggest cricket league, it usually takes place during the summer for 2 months, between March and May each year. It has an exclusive window in the ICC Future Tours Programme, resulting in fewer international cricket tours occurring during the IPL seasons.[5]\r\n\r\nIn 2014, it ranked sixth in average attendance among all sports leagues.[6] In 2010, the IPL became the first sporting event to be broadcast live on YouTube.[7][8] Inspired by the success of the IPL, other Indian sports leagues have been established.[a][11][12][13] In 2022, the league\'s brand value was estimated at ₹90,038 crore (US$11 billion).[14] According to the BCCI, the 2015 IPL season contributed ₹1,150 crore (US$1

### loading api and embedding

In [30]:
gemini_embed_model = GeminiEmbedding(model_name="models/embedding-001")

In [31]:
model = Gemini(models='gemini-pro',api_key=google_api_key)

> ServiceContext is deprecated it is replaced by Settings

In [32]:
Settings.llm = model
Settings.embed_model = gemini_embed_model
Settings.chunk_size = 800
Settings.chunk_overlap = 20


##### VectorStoreIndex

In [33]:
index = VectorStoreIndex.from_documents(
    doc, embed_model=gemini_embed_model
)

In [34]:
index.storage_context.persist()